# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [74]:
# Install required packages
!pip install requests

In [75]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [76]:
# Read data from CoinGecko API
url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
response = requests.get(url)
data = response.json()

data


{'bitcoin': {'usd': 121707}}

We will extract the Bitcoin price in USD and convert it to ZAR using an estimated exchange rate.
This is an example of turning raw data into useful **information**.

In [77]:
# Function to convert USD to ZAR
def convert_usd_to_zar(usd_amount, rate=18.5):
    return round(usd_amount * rate, 2)

In [78]:
# Extract USD price and apply transformation
usd_price = data["bitcoin"]["usd"]
zar_price = convert_usd_to_zar(usd_price)
timestamp = datetime.now().isoformat()

print(f"Bitcoin price in USD: {usd_price}")
print(f"Estimated price in ZAR: {zar_price}")


Bitcoin price in USD: 121707
Estimated price in ZAR: 2251579.5


In [79]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    -- These are SQL comments. Similarly /* ... */ for multi-line comments
    -- If timestamp should be unique (e.g., one price per moment):
    -- timestamp TEXT PRIMARY KEY,
    -- If multiple prices per second are expected, or if you want a simple unique ID:
    -- id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    usd REAL,
    zar REAL
)
''')

# Insert the new row
cursor.execute("INSERT INTO prices VALUES (?, ?, ?)", (timestamp, usd_price, zar_price))
conn.commit()
conn.close()


OperationalError: table prices has 4 columns but 3 values were supplied

In [ ]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()
